In [12]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from IPython.display import display, HTML

Load in the data from Natureserve

In [16]:
with open('data/natureserve-region-4-species-data.json') as json_file:
    data = json.load(json_file)

In [17]:
data[0]['speciesGlobal']

{'usesaCode': None,
 'cosewicCode': 'E',
 'saraCode': 'Endangered/En voie de disparition',
 'synonyms': ['Acris crepitans blanchardi'],
 'otherCommonNames': ["Blanchard's cricket frog",
  'Rainette grillon de Blanchard'],
 'kingdom': 'Animalia',
 'phylum': 'Craniata',
 'taxclass': 'Amphibia',
 'taxorder': 'Anura',
 'family': 'Hylidae',
 'genus': 'Acris',
 'taxonomicComments': 'Gamble et al. (2008) revised the distributions of <i>blanchardi </i>and <i>crepitans </i>in the south-central part of their combined ranges and recognized <i>A. blanchardi</i> and <i>A. crepitans </i>as distinct species. This change was adopted by Frost (Amphibian Species of the World website), Collins and Taggart (2009) and Crother (2012, 2017). The morphological distinctiveness of <i>Acris crepitans blanchardi</i> from <i>Acris crepitans crepitans</i> had been rejected by McCallum and Trauth (2006).',
 'informalTaxonomy': 'Animals | Vertebrates | Amphibians',
 'infraspecies': False,
 'completeDistribution': Tru

Get the data in the right format to turn it into a dataframe

In [18]:
for d in data:
    if 'speciesGlobal' in d:
        d['taxclass'] = d['speciesGlobal']['taxclass']
        d['taxorder'] = d['speciesGlobal']['taxorder']
        d['informal_taxonomy'] = d['speciesGlobal']['informalTaxonomy']
        
    else:
        d['taxclass'] = "NA"
        d['taxorder'] = "NA"
        d['informal_taxonomy'] = "NA"

In [19]:
for d in data:
    taxonomy = d['informal_taxonomy']
    split_taxonomy = taxonomy.split('|')
    d['category'] = split_taxonomy[0].strip()
    if len(split_taxonomy) == 3:
        d['sub_category'] = split_taxonomy[1].strip()
        d['type'] = split_taxonomy[2].strip()
    elif len(split_taxonomy) == 2:
        d['sub_category'] = split_taxonomy[1].strip()

Create a list of all the state abbreviations

In [20]:
state_abbreviations = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC","SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
len(state_abbreviations)

50

Reformat the data and remove the rows where there is no data.

In [21]:
for d in data:
    for nation in d['nations']:
        for states in nation['subnations']:
            # If the subnationCode is in the list of state abbreviations
            name = states['subnationCode']
            key = f"SRank_{name}"
            if states['subnationCode'] in state_abbreviations:
                d[key] = states['roundedSRank']
            else:
                pass
            
            key1 = f"native_{name}"
            if 'native' in states and states['native'] == True and states['subnationCode'] in state_abbreviations:
                d[key1] = True
            elif 'native' in states and states['native'] == False and states['subnationCode'] in state_abbreviations:
                d[key1] = False
            else:
                pass

In [22]:
# Turn data into a dataframe
df = pd.DataFrame(data)

In [23]:
df.to_csv('data/natureserve_wide.csv', index=False)

In [24]:
df = pd.read_csv('data/natureserve_wide.csv')

Filter down to freshwater species by keeping only the species where the informal_taxonomy is either "Crayfishes", "Fishes - Freshwater and Anadromous Bony, Cartilaginous; Lampreys", "Freshwater Mussels", "Freshwater Snails"

In [25]:
df = df[df['type'].isin(["Crayfishes", "Fishes - Freshwater and Anadromous Bony, Cartilaginous; Lampreys", "Freshwater Mussels", "Freshwater Snails"])]

In [26]:
df.groupby(['type']).count()

,elementGlobalId,uniqueId,nsxUrl,elcode,scientificName,formattedScientificName,primaryCommonName,primaryCommonNameLanguage,roundedGRank,nations,...,SRank_AZ,native_AZ,SRank_WY,native_WY,SRank_CA,native_CA,SRank_HI,native_HI,SRank_AK,native_AK
type,,,,,,,,,,,,,,,,,,,,,
Crayfishes,341,341,341,341,341,341,341,341,341,341,...,3,3,6,6,3,3,1,1,0,0
"Fishes - Freshwater and Anadromous Bony, Cartilaginous; Lampreys",666,666,666,666,666,666,666,666,666,666,...,62,62,59,59,39,39,9,9,11,11
Freshwater Mussels,275,275,275,275,275,275,275,275,275,275,...,1,1,7,7,0,0,0,0,0,0
Freshwater Snails,348,348,348,348,348,348,348,348,348,348,...,15,15,26,26,22,22,11,11,9,9


Turn the State Ranks into a numerical variable.

In [27]:
for column in df.columns[df.columns.str.startswith('SRank_')]:
    new_column = column.replace('SRank_', 'SRank_') + '_numerical'
    df[new_column] = df[column].replace({'S5': 5, 'S4': 4, 'S3': 3, 'S2': 2, 'S1': 1, 'SX': 0, 'SH': 0.5, 'SU': 0, 'SNR': float("nan"), 'SNA': float("nan"), 'S3N': 3, 'SHB': 0.5, 'S1N': 1, 'SHB,S1N': 1, 'SNRN': float("nan")})

In [28]:
for column in df.columns[df.columns.str.endswith('_numerical')]:
    df[column] = df[column].astype('float')

Here I create a new variable which is the highest state rank for each species. For each state, a species will now have the actual rank from that state and the highest rank of any of the states.

In [29]:
df['highest_srank'] = df[df.columns[df.columns.str.endswith('_numerical')]].max(axis=1)

In [30]:
df.groupby(['highest_srank'])['elementGlobalId'].count()

highest_srank
0.0     76
0.5     24
1.0    270
2.0    236
3.0    247
4.0    260
5.0    401
Name: elementGlobalId, dtype: int64

We need the data in long form to properly analyze it

In [31]:
df_long = pd.wide_to_long(df, stubnames=['SRank_','native_'], i=['scientificName', 'primaryCommonName', 'roundedGRank', 'gRank', 'taxclass', 'taxorder', 'category', 'sub_category', 'type', 'highest_srank'], j='state', suffix='\\w+')
df_long = df_long.reset_index()

But we are only interested in the information from states where a species actually occur, so we remove the row if the `SRank_` and `native_` is NA

In [32]:
df_long = df_long.dropna(subset=['SRank_', 'native_'])

In [33]:
df_long.to_csv('data/natureserve_long.csv', index=False)

Then we filter down the dataframe to only include Southeastern states

In this case, the Southeast is defined as the US Fish and Wildlife Services region 4: Alabama, Arkansas, Florida, Georiga, Kentucky, Louisiana, Mississippi, North Carolina, South Carolina and Tennessee.

In [34]:
df_long = df_long[df_long['state'].isin(['AL', 'AR', 'FL', 'GA', 'KY', 'LA', 'MS', 'NC', 'SC', 'TN'])]

In [35]:
df_long

,scientificName,primaryCommonName,roundedGRank,gRank,taxclass,taxorder,category,sub_category,type,highest_srank,...,primaryCommonNameLanguage,informal_taxonomy,speciesGlobal,nations,elementGlobalId,classificationStatus,formattedScientificName,nsxUrl,SRank_,native_
21,Acantharchus pomotis,Mud Sunfish,G4,G4G5,Actinopterygii,Perciformes,Animals,Vertebrates,"Fishes - Freshwater and Anadromous Bony, Carti...",4.0,...,EN,Animals | Vertebrates | Fishes - Freshwater an...,"{'usesaCode': None, 'cosewicCode': None, 'sara...","[{'nationCode': 'US', 'roundedNRank': 'N4', 's...",106210,Standard,<i>Acantharchus pomotis</i>,/Taxon/ELEMENT_GLOBAL.2.106210/Acantharchus_po...,S3,True
24,Acantharchus pomotis,Mud Sunfish,G4,G4G5,Actinopterygii,Perciformes,Animals,Vertebrates,"Fishes - Freshwater and Anadromous Bony, Carti...",4.0,...,EN,Animals | Vertebrates | Fishes - Freshwater an...,"{'usesaCode': None, 'cosewicCode': None, 'sara...","[{'nationCode': 'US', 'roundedNRank': 'N4', 's...",106210,Standard,<i>Acantharchus pomotis</i>,/Taxon/ELEMENT_GLOBAL.2.106210/Acantharchus_po...,S3,True
26,Acantharchus pomotis,Mud Sunfish,G4,G4G5,Actinopterygii,Perciformes,Animals,Vertebrates,"Fishes - Freshwater and Anadromous Bony, Carti...",4.0,...,EN,Animals | Vertebrates | Fishes - Freshwater an...,"{'usesaCode': None, 'cosewicCode': None, 'sara...","[{'nationCode': 'US', 'roundedNRank': 'N4', 's...",106210,Standard,<i>Acantharchus pomotis</i>,/Taxon/ELEMENT_GLOBAL.2.106210/Acantharchus_po...,S4,True
31,Acantharchus pomotis,Mud Sunfish,G4,G4G5,Actinopterygii,Perciformes,Animals,Vertebrates,"Fishes - Freshwater and Anadromous Bony, Carti...",4.0,...,EN,Animals | Vertebrates | Fishes - Freshwater an...,"{'usesaCode': None, 'cosewicCode': None, 'sara...","[{'nationCode': 'US', 'roundedNRank': 'N4', 's...",106210,Standard,<i>Acantharchus pomotis</i>,/Taxon/ELEMENT_GLOBAL.2.106210/Acantharchus_po...,S1,True
121,Acipenser brevirostrum,Shortnose Sturgeon,G3,G3,Actinopterygii,Acipenseriformes,Animals,Vertebrates,"Fishes - Freshwater and Anadromous Bony, Carti...",3.0,...,EN,Animals | Vertebrates | Fishes - Freshwater an...,"{'usesaCode': 'E', 'cosewicCode': 'SC', 'saraC...","[{'nationCode': 'US', 'roundedNRank': 'N3', 's...",105033,Standard,<i>Acipenser brevirostrum</i>,/Taxon/ELEMENT_GLOBAL.2.105033/Acipenser_brevi...,S1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162721,Procambarus youngi,Florida Longbeak Crayfish,G2,G2G3,Malacostraca,Decapoda,Animals,"Crayfish, Shrimp, & Other Crustaceans",Crayfishes,1.0,...,EN,"Animals | Crayfish, Shrimp, & Other Crustacean...","{'usesaCode': None, 'cosewicCode': None, 'sara...","[{'nationCode': 'US', 'roundedNRank': 'N2', 's...",118132,Standard,<i>Procambarus youngi</i>,/Taxon/ELEMENT_GLOBAL.2.118132/Procambarus_youngi,S1,True
162811,Procambarus zonangulus,Southern White River Crawfish,G5,G5,Malacostraca,Decapoda,Animals,"Crayfish, Shrimp, & Other Crustaceans",Crayfishes,5.0,...,EN,"Animals | Crayfish, Shrimp, & Other Crustacean...","{'usesaCode': None, 'cosewicCode': None, 'sara...","[{'nationCode': 'US', 'roundedNRank': 'N5', 's...",115455,Standard,<i>Procambarus zonangulus</i>,/Taxon/ELEMENT_GLOBAL.2.115455/Procambarus_zon...,S4,True
162812,Procambarus zonangulus,Southern White River Crawfish,G5,G5,Malacostraca,Decapoda,Animals,"Crayfish, Shrimp, & Other Crustaceans",Crayfishes,5.0,...,EN,"Animals | Crayfish, Shrimp, & Other Crustacean...","{'usesaCode': None, 'cosewicCode': None, 'sara...","[{'nationCode': 'US', 'roundedNRank': 'N5', 's...",115455,Standard,<i>Procambarus zonangulus</i>,/Taxon/ELEMENT_GLOBAL.2.115455/Procambarus_zon...,S5,True
162831,Procambarus zonangulus,Southern White River Crawfish,G5,G5,Malacostraca,Decapoda,Animals,"Crayfish, Shrimp, & Other Crustaceans",Crayfishes,5.0,...,EN,"Animals | Crayfish, Shrimp, & Other Crustacean...","{'usesaCode': None, 'cosewicCode': None, 'sara...","[{'nationCode': 'US', 'roundedNRank': 'N5', 's...",115455,Standard,<i>Proca

In [36]:
df_long.groupby(['state','SRank_']).count()

scientificName  primaryCommonName  roundedGRank  gRank  \
state SRank_                                                           
AL    S1                 172                172           172    172   
      S2                 130                130           130    130   
      S3                 120                120           120    120   
      S4                  75                 75            75     75   
      S5                 142                142           142    142   
...                      ...                ...           ...    ...   
TN    SH                   7                  7             7      7   
      SNA                 15                 15            15     15   
      SNR                 41                 41            41     41   
      SU                   5                  5             5      5   
      SX                  22                 22            22     22   

              taxclass  taxorder  category  sub_category  type  highest_srank  \
state SRank_                                                                    
AL    S1           172       172       172           172   172            172   
      S2           130       130       130           130   130            130   
      S3           120       120       120           120   120            120   
      S4            75        75        75            75    75             75   
      S5           142       142       142           142   142            142   
...                ...       ...       ...           ...   ...            ...   
TN    SH             7         7         7             7     7              7   
      SNA           15        15        15            15    15             11   
      SNR           41        41        41            41    41             25   
      SU             5         5         5             5     5              5   
      SX            22        22        22            22    22             22   

              ...  uniqueId  primaryCommonNameLanguage  informal_taxonomy  \
state SRank_  ...                                                           
AL    S1      ...       172                        172                172   
      S2      ...       130                        130                130   
      S3      ...       120                        120                120   
      S4      ...        75                         75                 75   
      S5      ...       142                        142                142   
...           ...       ...                        ...                ...   
TN    SH      ...         7                          7                  7   
      SNA     ...        15                         15                 15   
      SNR     ...        41                         41                 41   
      SU      ...         5                          5                  5   
      SX      ...        22                         22                 22   

              speciesGlobal  nations  elementGlobalId  classificationStatus  \
state SRank_                                                                  
AL    S1                172      172              172                   172   
      S2                130      130              130                   130   
      S3                120      120              120                   120   
      S4                 75       75               75                    75   
      S5                142      142              142                   142   
...                     ...      ...              ...                   ...   
TN    SH                  7        7                7                     7   
      SNA                15       15               15                    15   
      SNR                41       41               41                    41   
      SU                  5        5                5                     5   
      SX                 22       22               22                    

In [37]:
df_long.groupby(['scientificName','highest_srank'])['SRank_'].max()

scientificName           highest_srank
Acantharchus pomotis     4.0               S4
Acipenser brevirostrum   3.0               S3
Acipenser fulvescens     3.0               S3
Acipenser oxyrinchus     3.0              SNR
Actinonaias ligamentina  5.0               S5
                                         ... 
Villosa vanuxemensis     4.0               S4
Villosa vaughaniana      3.0               S3
Villosa vibex            5.0              SNR
Villosa villosa          3.0               S3
Vitta virginea           3.0               S3
Name: SRank_, Length: 1514, dtype: object

In [38]:
grouped = df_long.groupby('highest_srank')['scientificName'].count().reset_index()
grouped['pct'] = round(grouped['scientificName'] / grouped['scientificName'].sum() * 100)
grouped


,highest_srank,scientificName,pct
0,0.0,122,3.0
1,0.5,32,1.0
2,1.0,416,9.0
3,2.0,465,10.0
4,3.0,525,12.0
5,4.0,733,16.0
6,5.0,2207,49.0


In [39]:
type_grouped = df_long.groupby(['type','highest_srank'])['scientificName'].count().reset_index()
type_grouped['pct'] = type_grouped['scientificName'] / type_grouped.groupby('type')['scientificName'].transform('sum') * 100
type_grouped

,type,highest_srank,scientificName,pct
0,Crayfishes,0.0,1,0.162866
1,Crayfishes,1.0,70,11.400651
2,Crayfishes,2.0,108,17.589577
3,Crayfishes,3.0,122,19.869707
4,Crayfishes,4.0,126,20.521173
5,Crayfishes,5.0,187,30.456026
6,"Fishes - Freshwater and Anadromous Bony, Carti...",0.0,7,0.304613
7,"Fishes - Freshwater and Anadromous Bony, Carti...",1.0,86,3.742385
8,"Fishes - Freshwater and Anadromous Bony, Carti...",2.0,127,5.526545
9,"Fishes - Freshwater and Anadromous Bony, Carti...",3.0,201,8.746736


In [40]:
type_grouped.to_clipboard()

In [ ]:
species_types = df_long.groupby('type')['scientificName'].nunique().reset_index()
species_types

,type,scientificName
0,Crayfishes,341
1,"Fishes - Freshwater and Anadromous Bony, Carti...",666
2,Freshwater Mussels,275
3,Freshwater Snails,348


In [ ]:
state_grouped = df_long.groupby(['state','highest_srank'])['scientificName'].count().reset_index()
state_grouped['highest_rank_pct'] = state_grouped['scientificName'] / state_grouped.groupby('state')['scientificName'].transform('sum') * 100
state_grouped.head(50)

,state,highest_srank,scientificName,highest_rank_pct
0,AL,0.0,64,7.990012
1,AL,0.5,19,2.372035
2,AL,1.0,123,15.355805
3,AL,2.0,98,12.234707
4,AL,3.0,102,12.734082
5,AL,4.0,107,13.358302
6,AL,5.0,288,35.955056
7,AR,0.0,4,1.028278
8,AR,1.0,22,5.655527
9,AR,2.0,35,8.997429


In [ ]:
state_grouped_wide = state_grouped.pivot(index='state', columns='highest_srank', values='scientificName').reset_index()
state_grouped_wide

highest_srank,state,0.0,0.5,1.0,2.0,3.0,4.0,5.0
0,AL,64.0,19.0,123.0,98.0,102.0,107.0,288.0
1,AR,4.0,NaN,22.0,35.0,56.0,60.0,212.0
2,FL,2.0,4.0,49.0,40.0,42.0,37.0,160.0
3,GA,11.0,5.0,49.0,69.0,86.0,91.0,253.0
4,KY,15.0,1.0,32.0,38.0,39.0,94.0,253.0
5,LA,NaN,NaN,6.0,21.0,29.0,41.0,200.0
6,MS,4.0,1.0,18.0,42.0,34.0,65.0,242.0
7,NC,2.0,NaN,25.0,34.0,49.0,75.0,201.0
8,SC,1.0,NaN,8.0,15.0,28.0,53.0,138.0
9,TN,19.0,2.0,84.0,73.0,60.0,110.0,260.0


In [ ]:
state_grouped_wide_pct = state_grouped.pivot(index='state', columns='highest_srank', values='highest_rank_pct').reset_index()
state_grouped_wide_pct

highest_srank,state,0.0,0.5,1.0,2.0,3.0,4.0,5.0
0,AL,7.990012,2.372035,15.355805,12.234707,12.734082,13.358302,35.955056
1,AR,1.028278,NaN,5.655527,8.997429,14.395887,15.424165,54.498715
2,FL,0.598802,1.197605,14.670659,11.976048,12.574850,11.077844,47.904192
3,GA,1.950355,0.886525,8.687943,12.234043,15.248227,16.134752,44.858156
4,KY,3.177966,0.211864,6.779661,8.050847,8.262712,19.915254,53.601695
5,LA,NaN,NaN,2.020202,7.070707,9.764310,13.804714,67.340067
6,MS,0.985222,0.246305,4.433498,10.344828,8.374384,16.009852,59.605911
7,NC,0.518135,NaN,6.476684,8.808290,12.694301,19.430052,52.072539
8,SC,0.411523,NaN,3.292181,6.172840,11.522634,21.810700,56.790123
9,TN,3.125000,0.328947,13.815789,12.006579,9.868421,18.092105,42.763158


In [ ]:
state_grouped1 = df_long.groupby(['state','SRank_'])['scientificName'].count().reset_index()
state_grouped1['s_rank_pct'] = state_grouped1['scientificName'] / state_grouped1.groupby('state')['scientificName'].transform('sum') * 100
state_grouped1.head(50)

,state,SRank_,scientificName,s_rank_pct
0,AL,S1,172,20.798065
1,AL,S2,130,15.719468
2,AL,S3,120,14.510278
3,AL,S4,75,9.068924
4,AL,S5,142,17.170496
5,AL,SH,19,2.297461
6,AL,SNA,21,2.539299
7,AL,SNR,44,5.320435
8,AL,SU,4,0.483676
9,AL,SX,100,12.091898


In [ ]:
state_grouped1_wide = state_grouped1.pivot(index='state', columns='SRank_', values='scientificName').reset_index()
state_grouped1_wide

SRank_,state,S1,S2,S3,S4,S5,SH,SNA,SNR,SU,SX
0,AL,172.0,130.0,120.0,75.0,142.0,19.0,21.0,44.0,4.0,100.0
1,AR,41.0,51.0,105.0,111.0,5.0,7.0,20.0,58.0,4.0,4.0
2,FL,77.0,36.0,23.0,17.0,32.0,6.0,54.0,152.0,NaN,1.0
3,GA,85.0,87.0,106.0,66.0,92.0,12.0,27.0,67.0,11.0,28.0
4,KY,74.0,44.0,36.0,201.0,10.0,NaN,27.0,65.0,15.0,24.0
5,LA,27.0,34.0,36.0,66.0,95.0,2.0,12.0,35.0,2.0,NaN
6,MS,55.0,50.0,51.0,52.0,130.0,6.0,14.0,56.0,2.0,6.0
7,NC,64.0,56.0,64.0,55.0,82.0,2.0,35.0,18.0,15.0,11.0
8,SC,22.0,25.0,42.0,28.0,7.0,2.0,18.0,107.0,3.0,NaN
9,TN,115.0,85.0,74.0,94.0,170.0,7.0,15.0,41.0,5.0,22.0


In [ ]:
state_grouped1_wide_pct = state_grouped1.pivot(index='state', columns='SRank_', values='s_rank_pct').reset_index()
state_grouped1_wide_pct

SRank_,state,S1,S2,S3,S4,S5,SH,SNA,SNR,SU,SX
0,AL,20.798065,15.719468,14.510278,9.068924,17.170496,2.297461,2.539299,5.320435,0.483676,12.091898
1,AR,10.098522,12.561576,25.862069,27.339901,1.231527,1.724138,4.926108,14.285714,0.985222,0.985222
2,FL,19.346734,9.045226,5.778894,4.271357,8.040201,1.507538,13.567839,38.190955,NaN,0.251256
3,GA,14.629948,14.974182,18.244406,11.359725,15.834768,2.065404,4.647160,11.531842,1.893287,4.819277
4,KY,14.919355,8.870968,7.258065,40.524194,2.016129,NaN,5.443548,13.104839,3.024194,4.838710
5,LA,8.737864,11.003236,11.650485,21.359223,30.744337,0.647249,3.883495,11.326861,0.647249,NaN
6,MS,13.033175,11.848341,12.085308,12.322275,30.805687,1.421801,3.317536,13.270142,0.473934,1.421801
7,NC,15.920398,13.930348,15.920398,13.681592,20.398010,0.497512,8.706468,4.477612,3.731343,2.736318
8,SC,8.661417,9.842520,16.535433,11.023622,2.755906,0.787402,7.086614,42.125984,1.181102,NaN
9,TN,18.312102,13.535032,11.783439,14.968153,27.070064,1.114650,2.388535,6.528662,0.796178,3.503185


In [ ]:
df_long.groupby('SRank_')['SRank_'].count()

SRank_
S1     732
S2     598
S3     657
S4     765
S5     765
SH      63
SNA    243
SNR    643
SU      61
SX     196
Name: SRank_, dtype: int64

In [ ]:
filtered_df = df_long.loc[df_long['SRank_'].isin(['S1','S2','S3','S4','S5'])]
filtered_state = filtered_df.groupby(['state','SRank_'])['scientificName'].count().reset_index()
filtered_state['s_rank_pct'] = filtered_state['scientificName'] / filtered_state.groupby('state')['scientificName'].transform('sum') * 100
filtered_state

,state,SRank_,scientificName,s_rank_pct
0,AL,S1,172,26.917058
1,AL,S2,130,20.344288
2,AL,S3,120,18.779343
3,AL,S4,75,11.737089
4,AL,S5,142,22.222222
5,AR,S1,41,13.099042
6,AR,S2,51,16.293930
7,AR,S3,105,33.546326
8,AR,S4,111,35.463259
9,AR,S5,5,1.597444


In [ ]:
al = df_long[df_long['state'].isin(['AL'])]
fl = df_long[df_long['state'].isin(['FL'])]
ga = df_long[df_long['state'].isin(['GA'])]
ms = df_long[df_long['state'].isin(['MS'])]
tn = df_long[df_long['state'].isin(['TN'])]

### Alabama

In [ ]:
al.head(3)

,scientificName,primaryCommonName,roundedGRank,gRank,taxclass,taxorder,category,sub_category,type,highest_srank,state,elcode,speciesGlobal,informal_taxonomy,nsxUrl,primaryCommonNameLanguage,elementGlobalId,lastModified,classificationStatus,nations,uniqueId,formattedScientificName,SRank_,native_
31,Acantharchus pomotis,Mud Sunfish,G4,G4G5,Actinopterygii,Perciformes,Animals,Vertebrates,"Fishes - Freshwater and Anadromous Bony, Carti...",4.0,AL,AFCQB05010,"{'usesaCode': None, 'cosewicCode': None, 'sara...",Animals | Vertebrates | Fishes - Freshwater an...,/Taxon/ELEMENT_GLOBAL.2.106210/Acantharchus_po...,EN,106210,2023-06-02T21:52:52.693642Z,Standard,"[{'nationCode': 'US', 'roundedNRank': 'N4', 's...",ELEMENT_GLOBAL.2.106210,<i>Acantharchus pomotis</i>,S1,True
231,Acipenser fulvescens,Lake Sturgeon,G3,G3G4,Actinopterygii,Acipenseriformes,Animals,Vertebrates,"Fishes - Freshwater and Anadromous Bony, Carti...",3.0,AL,AFCAA01020,"{'usesaCode': 'UR', 'cosewicCode': None, 'sara...",Animals | Vertebrates | Fishes - Freshwater an...,/Taxon/ELEMENT_GLOBAL.2.104232/Acipenser_fulve...,EN,104232,2023-06-02T21:48:17.776373Z,Standard,"[{'nationCode': 'CA', 'roundedNRank': 'N3', 's...",ELEMENT_GLOBAL.2.104232,<i>Acipenser fulvescens</i>,S1,True
331,Acipenser oxyrinchus,Atlantic Sturgeon,G3,G3,Actinopterygii,Acipenseriformes,Animals,Vertebrates,"Fishes - Freshwater and Anadromous Bony, Carti...",3.0,AL,AFCAA01040,"{'usesaCode': None, 'cosewicCode': None, 'sara...",Animals | Vertebrates | Fishes - Freshwater an...,/Taxon/ELEMENT_GLOBAL.2.102787/Acipenser_oxyri...,EN,102787,2023-06-02T21:44:31.305522Z,Standard,"[{'nationCode': 'US', 'roundedNRank': 'N3', 's...",ELEMENT_GLOBAL.2.102787,<i>Acipenser oxyrinchus</i>,S2,True


In [ ]:
al_cross = pd.crosstab(al.highest_srank, al.SRank_)
al_cross

SRank_,S1,S2,S3,S4,S5,SH,SNA,SNR,SU,SX
highest_srank,,,,,,,,,,
0.0,0,0,0,0,0,0,0,0,4,60
0.5,0,0,0,0,0,15,0,0,0,4
1.0,96,0,0,0,0,3,0,4,0,20
2.0,23,63,0,0,0,0,1,2,0,9
3.0,17,18,63,0,0,0,0,3,0,1
4.0,18,15,17,45,0,1,0,7,0,4
5.0,18,34,40,30,142,0,7,15,0,2


In [ ]:
al_grouped = al.groupby(['type','SRank_'])['scientificName'].count().reset_index()

al_grouped['pct'] = al_grouped['scientificName'] / al_grouped.groupby('type')['scientificName'].transform('sum') * 100
al_grouped['pct'] = round(al_grouped['pct'], 2)
al_grouped

,type,SRank_,scientificName,pct
0,Crayfishes,S1,25,25.00
1,Crayfishes,S2,26,26.00
2,Crayfishes,S3,22,22.00
3,Crayfishes,S4,11,11.00
4,Crayfishes,S5,8,8.00
5,Crayfishes,SNA,2,2.00
6,Crayfishes,SNR,6,6.00
7,"Fishes - Freshwater and Anadromous Bony, Carti...",S1,45,13.47
8,"Fishes - Freshwater and Anadromous Bony, Carti...",S2,48,14.37
9,"Fishes - Freshwater and Anadromous Bony, Carti...",S3,57,17.07


### Florida

In [ ]:
fl_cross = pd.crosstab(fl.highest_srank, fl.SRank_)
fl_cross

SRank_,S1,S2,S3,S4,S5,SH,SNA,SNR,SX
highest_srank,,,,,,,,,
0.0,0,0,0,0,0,0,0,1,1
0.5,0,0,0,0,0,4,0,0,0
1.0,47,0,0,0,0,0,0,2,0
2.0,14,20,0,0,0,1,0,5,0
3.0,6,9,16,0,0,0,0,11,0
4.0,2,3,3,10,0,0,0,19,0
5.0,8,4,4,7,32,1,6,98,0


In [ ]:
fl_grouped = fl.groupby(['type','SRank_'])['scientificName'].count().reset_index()
fl_grouped['pct'] = fl_grouped['scientificName'] / fl_grouped.groupby('type')['scientificName'].transform('sum') * 100
fl_grouped['pct'] = round(fl_grouped['pct'], 2)

In [ ]:
fl_grouped

,type,SRank_,scientificName,pct
0,Crayfishes,S1,19,33.93
1,Crayfishes,S2,12,21.43
2,Crayfishes,S3,5,8.93
3,Crayfishes,S4,11,19.64
4,Crayfishes,S5,3,5.36
5,Crayfishes,SNR,6,10.71
6,"Fishes - Freshwater and Anadromous Bony, Carti...",S1,14,7.18
7,"Fishes - Freshwater and Anadromous Bony, Carti...",S2,13,6.67
8,"Fishes - Freshwater and Anadromous Bony, Carti...",S3,9,4.62
9,"Fishes - Freshwater and Anadromous Bony, Carti...",S4,4,2.05


### Tennessee

In [ ]:
tn_cross = pd.crosstab(tn.highest_srank, tn.SRank_)
tn_cross

SRank_,S1,S2,S3,S4,S5,SH,SNA,SNR,SU,SX
highest_srank,,,,,,,,,,
0.0,0,0,0,0,0,0,1,0,0,18
0.5,0,0,0,0,0,2,0,0,0,0
1.0,79,0,0,0,0,2,0,2,0,1
2.0,15,51,0,0,0,3,0,2,0,2
3.0,12,12,33,0,0,0,0,3,0,0
4.0,4,12,25,60,0,0,0,6,3,0
5.0,5,10,16,34,170,0,10,12,2,1


In [ ]:
tn_grouped = tn.groupby(['type','SRank_'])['scientificName'].count().reset_index()
tn_grouped['pct'] = tn_grouped['scientificName'] / tn_grouped.groupby('type')['scientificName'].transform('sum') * 100
tn_grouped['pct'] = round(tn_grouped['pct'], 2)

tn_grouped

,type,SRank_,scientificName,pct
0,Crayfishes,S1,16,17.20
1,Crayfishes,S2,9,9.68
2,Crayfishes,S3,12,12.90
3,Crayfishes,S4,20,21.51
4,Crayfishes,S5,24,25.81
5,Crayfishes,SNA,1,1.08
6,Crayfishes,SNR,11,11.83
7,"Fishes - Freshwater and Anadromous Bony, Carti...",S1,42,13.38
8,"Fishes - Freshwater and Anadromous Bony, Carti...",S2,39,12.42
9,"Fishes - Freshwater and Anadromous Bony, Carti...",S3,45,14.33


In [ ]:
type_grouped = df.groupby(['type','highest_srank'])['scientificName'].count().reset_index()
type_grouped['highest_rank_pct'] = type_grouped['scientificName'] / type_grouped.groupby('type')['scientificName'].transform('sum') * 100
type_grouped

,type,highest_srank,scientificName,highest_rank_pct
0,Crayfishes,0.0,1,0.309598
1,Crayfishes,1.0,62,19.195046
2,Crayfishes,2.0,76,23.529412
3,Crayfishes,3.0,79,24.458204
4,Crayfishes,4.0,58,17.956656
5,Crayfishes,5.0,47,14.551084
6,"Fishes - Freshwater and Anadromous Bony, Carti...",0.0,3,0.503356
7,"Fishes - Freshwater and Anadromous Bony, Carti...",1.0,54,9.060403
8,"Fishes - Freshwater and Anadromous Bony, Carti...",2.0,66,11.073826
9,"Fishes - Freshwater and Anadromous Bony, Carti...",3.0,88,14.765101


## Dams

In [ ]:
dams = pd.read_csv("data/dams.csv")

In [ ]:
dams

,Dam Name,Other Names,Former Names,NID ID,Other Structure ID,Federal ID,Owner Names,Owner Types,Primary Owner Type,Number of Associated Structures,Designer Names,Non-Federal Dam on Federal Property,Primary Purpose,Purposes,Source Agency,State or Federal Agency ID,Latitude,Longitude,State,County,City,Distance to Nearest City (Miles),River or Stream Name,Congressional District,"Section, Township, or Range Location",State Regulated Dam,State Jurisdictional Dam,State Regulatory Agency,State Permitting Authority,State Inspection Authority,State Enforcement Authority,Federal Agency Owners,Federal Agency Involvement Funding,Federal Agency Involvement Design,Federal Agency Involvement Construction,Federal Agency Involvement Regulatory,Federal Agency Involvement Inspection,Federal Agency Involvement Operation,Federal Agency Involvement Other,Primary Dam Type,Dam Types,Core Types,Foundation,Dam Height (Ft),Hydraulic Height (Ft),Structural Height (Ft),NID Height (Ft),NID Height Category,Dam Length (Ft),Volume (Cubic Yards),Year Completed,Year Completed Category,Years Modified,NID Storage (Acre-Ft),Max Storage (Acre-Ft),Normal Storage (Acre-Ft),Surface Area (Acres),Drainage Area (Sq Miles),Max Discharge (Cubic Ft/Second),Spillway Type,Spillway Width (Ft),Number of Locks,Length of Locks,Length of Secondary Lock,Lock Width (Ft),Secondary Lock Width (Ft),Outlet Gate Type,Data Last Updated,Last Inspection Date,Inspection Frequency,Hazard Potential Classification,Condition Assessment,Condition Assessment Date,EAP Prepared,EAP Last Revision Date,Website URL,American Indian/Alaska Native/Native Hawaiian
0,Eugene J Burrell Lock and Dam,Burrell Lock and Dam,NaN,FL00708,NaN,FL00708,LAKE COUNTY WATER AUTHORITY,State,State,0.0,NaN,No,Flood Risk Reduction,Flood Risk Reduction;Navigation,Florida,NaN,28.871489,-81.783317,Florida,Lake,LISBON,0.0,HAINES CREEK,"Congressional District 06, Florida","S2, T19S, R25E",Yes,Yes,SJRWMD,Yes,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Earth,Earth;Other,NaN,NaN,19.0,19.0,14.0,19.0,Less than 25 feet,500.0,88000.0,1957.0,1950-1959,NaN,88000.0,88000.0,80000.0,1105.0,0.00,4200.0,NaN,0.0,0.0,0.0,NaN,0.0,NaN,NaN,2023-05-04,2014-08-12,NaN,High,Satisfactory,2014-08-12,No,NaN,https://damsafety.org/florida,NaN
1,Huneycutt Pig Farm Dam,NaN,NaN,NC06252,NaN,NC06252,NaN,Not Listed,Not Listed,NaN,NaN,No,Fish and Wildlife Pond,Fish and Wildlife Pond,North Carolina,STANL-384,35.349500,-80.302700,North Carolina,Stanly,Albemarle,NaN,Trib. to Ramsey Ck,"Congressional District 08, North Carolina",NaN,Yes,Yes,"NCDEQ, DEMLR, Dam Safety Program",Yes,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Earth,NaN,NaN,NaN,11.0,15.0,15.0,Less than 25 feet,309.0,NaN,NaN,Undetermined,NaN,29.0,29.0,17.0,3.4,31.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-09,2021-03-02,2.0,High,Fair,2021-03-02,Yes,2018-07-10,https://damsafety.org/north-carolina,NaN
2,Billy Barlow Dam,Barlow Pond,NaN,GA05699,NaN,GA05699,Billy Barlow,Private,Private,NaN,NaN,NaN,Fish and Wildlife Pond,Fish and Wildlife Pond;Irrigation,Natural Resources Conservation Service,GAA023-011,32.467582,-83.309279,Georgia,Bleckley,Lumber City,64.0,Tr- Gully Branch,"Congressional District 08, Georgia",NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Natural Resources Conservation Service;Natural...,Natural Resources Conservation Service,NaN,NaN,NaN,NaN,Earth,Earth,NaN,NaN,19.0,0.0,0.0,19.0,Less than 25 feet,1200.0,11900.0,1978.0,1970-1979,NaN,88.0,88.0,48.0,0.0,0.19,210.0,Uncontrolled,45.0,NaN,NaN,NaN,NaN,NaN,NaN,2021-05-24,NaN,0.0,Low,Not Available,NaN,No,NaN,http://www.damsafety.info/,NaN
3,Ellijay River W/S Str. #11 Dam,NaN,NaN,GA02192,NaN,GA02192,Gilmer County Board of Commissoners;Limestone ...,State;Local Government,State,0.0,SCS,No,Flood Risk Reduction,Flood Risk Reduction,Georgia,061-009-00649,34.826240,-84.368470,Georgia,Gilmer,NaN,NaN,SISSON CREEK,"Congressional District 09, Georgia",NaN,Yes,Yes,GA-SAFE DAMS PROGRAM,Yes,Yes,Yes,NaN,NaN,Natural Resources Conservation Service,NaN,NaN,NaN,NaN,NaN,Earth,Earth,Unli

In [ ]:
dams_19 = dams.loc[dams['Year Completed'] > 1899.0]

In [ ]:
dams_19

,Dam Name,Other Names,Former Names,NID ID,Other Structure ID,Federal ID,Owner Names,Owner Types,Primary Owner Type,Number of Associated Structures,Designer Names,Non-Federal Dam on Federal Property,Primary Purpose,Purposes,Source Agency,State or Federal Agency ID,Latitude,Longitude,State,County,City,Distance to Nearest City (Miles),River or Stream Name,Congressional District,"Section, Township, or Range Location",State Regulated Dam,State Jurisdictional Dam,State Regulatory Agency,State Permitting Authority,State Inspection Authority,State Enforcement Authority,Federal Agency Owners,Federal Agency Involvement Funding,Federal Agency Involvement Design,Federal Agency Involvement Construction,Federal Agency Involvement Regulatory,Federal Agency Involvement Inspection,Federal Agency Involvement Operation,Federal Agency Involvement Other,Primary Dam Type,Dam Types,Core Types,Foundation,Dam Height (Ft),Hydraulic Height (Ft),Structural Height (Ft),NID Height (Ft),NID Height Category,Dam Length (Ft),Volume (Cubic Yards),Year Completed,Year Completed Category,Years Modified,NID Storage (Acre-Ft),Max Storage (Acre-Ft),Normal Storage (Acre-Ft),Surface Area (Acres),Drainage Area (Sq Miles),Max Discharge (Cubic Ft/Second),Spillway Type,Spillway Width (Ft),Number of Locks,Length of Locks,Length of Secondary Lock,Lock Width (Ft),Secondary Lock Width (Ft),Outlet Gate Type,Data Last Updated,Last Inspection Date,Inspection Frequency,Hazard Potential Classification,Condition Assessment,Condition Assessment Date,EAP Prepared,EAP Last Revision Date,Website URL,American Indian/Alaska Native/Native Hawaiian
0,Eugene J Burrell Lock and Dam,Burrell Lock and Dam,NaN,FL00708,NaN,FL00708,LAKE COUNTY WATER AUTHORITY,State,State,0.0,NaN,No,Flood Risk Reduction,Flood Risk Reduction;Navigation,Florida,NaN,28.871489,-81.783317,Florida,Lake,LISBON,0.0,HAINES CREEK,"Congressional District 06, Florida","S2, T19S, R25E",Yes,Yes,SJRWMD,Yes,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Earth,Earth;Other,NaN,NaN,19.0,19.0,14.0,19.0,Less than 25 feet,500.0,88000.0,1957.0,1950-1959,NaN,88000.0,88000.0,80000.0,1105.0,0.00,4200.0,NaN,0.0,0.0,0.0,NaN,0.0,NaN,NaN,2023-05-04,2014-08-12,NaN,High,Satisfactory,2014-08-12,No,NaN,https://damsafety.org/florida,NaN
2,Billy Barlow Dam,Barlow Pond,NaN,GA05699,NaN,GA05699,Billy Barlow,Private,Private,NaN,NaN,NaN,Fish and Wildlife Pond,Fish and Wildlife Pond;Irrigation,Natural Resources Conservation Service,GAA023-011,32.467582,-83.309279,Georgia,Bleckley,Lumber City,64.0,Tr- Gully Branch,"Congressional District 08, Georgia",NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Natural Resources Conservation Service;Natural...,Natural Resources Conservation Service,NaN,NaN,NaN,NaN,Earth,Earth,NaN,NaN,19.0,0.0,0.0,19.0,Less than 25 feet,1200.0,11900.0,1978.0,1970-1979,NaN,88.0,88.0,48.0,0.0,0.19,210.0,Uncontrolled,45.0,NaN,NaN,NaN,NaN,NaN,NaN,2021-05-24,NaN,0.0,Low,Not Available,NaN,No,NaN,http://www.damsafety.info/,NaN
3,Ellijay River W/S Str. #11 Dam,NaN,NaN,GA02192,NaN,GA02192,Gilmer County Board of Commissoners;Limestone ...,State;Local Government,State,0.0,SCS,No,Flood Risk Reduction,Flood Risk Reduction,Georgia,061-009-00649,34.826240,-84.368470,Georgia,Gilmer,NaN,NaN,SISSON CREEK,"Congressional District 09, Georgia",NaN,Yes,Yes,GA-SAFE DAMS PROGRAM,Yes,Yes,Yes,NaN,NaN,Natural Resources Conservation Service,NaN,NaN,NaN,NaN,NaN,Earth,Earth,Unlisted/Unknown,Unlisted/Unknown,46.0,NaN,NaN,46.0,25-50 feet,231.0,NaN,1967.0,1960-1969,NaN,429.0,429.0,25.0,6.0,909.00,NaN,Uncontrolled,NaN,NaN,NaN,NaN,NaN,NaN,Uncontrolled,2021-05-24,2016-03-01,2.0,High,Poor,2016-03-01,Yes,NaN,https://damsafety.org/georgia,NaN
4,Marvin Lewis Pond,NaN,NaN,GA05711,NaN,GA05711,Marvin Lewis,Private,Private,NaN,NaN,NaN,Fish and Wildlife Pond,Fish and Wildlife Pond;Irrigation,Natural Resources Conservation Service,GAA081-002,31.881449,-83.731006,Georgia,Crisp,None,0.0,Tr- Lime Creek,"Congressional District 08, Georgia",NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Natural Resources Conservation Ser

### What kind of dams?

In [ ]:
dams.groupby('Primary Owner Type')['Dam Name'].count().reset_index()

,Primary Owner Type,Dam Name
0,Federal,449
1,Local Government,2121
2,Not Listed,409
3,Private,18840
4,Public Utility,271
5,State,1020


In [ ]:
dams.groupby('Year Completed Category')['Dam Name'].count().reset_index()

,Year Completed Category,Dam Name
0,1900-1909,180
1,1910-1919,115
2,1920-1929,242
3,1930-1939,405
4,1940-1949,852
5,1950-1959,2839
6,1960-1969,4103
7,1970-1979,2714
8,1980-1989,1280
9,1990-1999,1000


In [ ]:
dams1 = dams.groupby(['Primary Owner Type','NID Height Category'])['Dam Name'].count().reset_index()

In [ ]:
dams1.sort_values('Dam Name', ascending=False)

,Primary Owner Type,NID Height Category,Dam Name
17,Private,Less than 25 feet,12584
14,Private,25-50 feet,5588
4,Local Government,25-50 feet,1160
7,Local Government,Less than 25 feet,752
24,State,25-50 feet,519
15,Private,51-100 feet,407
27,State,Less than 25 feet,379
12,Not Listed,Less than 25 feet,202
5,Local Government,51-100 feet,191
16,Private,Greater than 100 feet,163


In [ ]:
large_dams = dams.loc[dams['Dam Height (Ft)'] > 49.9]
large_dams

,Dam Name,Other Names,Former Names,NID ID,Other Structure ID,Federal ID,Owner Names,Owner Types,Primary Owner Type,Number of Associated Structures,Designer Names,Non-Federal Dam on Federal Property,Primary Purpose,Purposes,Source Agency,State or Federal Agency ID,Latitude,Longitude,State,County,City,Distance to Nearest City (Miles),River or Stream Name,Congressional District,"Section, Township, or Range Location",State Regulated Dam,State Jurisdictional Dam,State Regulatory Agency,State Permitting Authority,State Inspection Authority,State Enforcement Authority,Federal Agency Owners,Federal Agency Involvement Funding,Federal Agency Involvement Design,Federal Agency Involvement Construction,Federal Agency Involvement Regulatory,Federal Agency Involvement Inspection,Federal Agency Involvement Operation,Federal Agency Involvement Other,Primary Dam Type,Dam Types,Core Types,Foundation,Dam Height (Ft),Hydraulic Height (Ft),Structural Height (Ft),NID Height (Ft),NID Height Category,Dam Length (Ft),Volume (Cubic Yards),Year Completed,Year Completed Category,Years Modified,NID Storage (Acre-Ft),Max Storage (Acre-Ft),Normal Storage (Acre-Ft),Surface Area (Acres),Drainage Area (Sq Miles),Max Discharge (Cubic Ft/Second),Spillway Type,Spillway Width (Ft),Number of Locks,Length of Locks,Length of Secondary Lock,Lock Width (Ft),Secondary Lock Width (Ft),Outlet Gate Type,Data Last Updated,Last Inspection Date,Inspection Frequency,Hazard Potential Classification,Condition Assessment,Condition Assessment Date,EAP Prepared,EAP Last Revision Date,Website URL,American Indian/Alaska Native/Native Hawaiian
7,Sautee Creek W/S Str. #22,NaN,NaN,GA00657,NaN,GA00657,Upper Chattahoochee River SWCD,State;Local Government,State,0.0,SCS,No,Flood Risk Reduction,Flood Risk Reduction,Georgia,068-016-01467,34.690000,-83.622850,Georgia,Habersham,SAUTEE,NaN,ROGERS CREEK,"Congressional District 09, Georgia",NaN,Yes,Yes,GA-SAFE DAMS PROGRAM,Yes,Yes,Yes,NaN,NaN,Natural Resources Conservation Service,NaN,NaN,NaN,NaN,NaN,Earth,Earth,Unlisted/Unknown,Unlisted/Unknown,50.0,43.0,43.0,50.0,51-100 feet,180.0,NaN,1960.0,1960-1969,NaN,573.0,573.0,54.0,6.0,576.0,NaN,Uncontrolled,NaN,NaN,NaN,NaN,NaN,NaN,Uncontrolled,2021-05-24,2017-01-11,2.0,High,Fair,2017-01-11,Yes,NaN,https://damsafety.org/georgia,NaN
18,Sharp Mountain Creek WS Str # 12,NaN,NaN,GA00692,NaN,GA00692,Limestone Valley SWCD,State,State,0.0,SCS,No,Flood Risk Reduction,Flood Risk Reduction,Georgia,112-030-02272,34.439260,-84.442100,Georgia,Pickens,COUNTY LINE COMMUNITY,NaN,TOWN CREEK,"Congressional District 11, Georgia",NaN,Yes,Yes,GA-SAFE DAMS PROGRAM,Yes,Yes,Yes,NaN,NaN,Natural Resources Conservation Service,NaN,NaN,NaN,NaN,NaN,Earth,Earth,Unlisted/Unknown,Unlisted/Unknown,67.0,NaN,NaN,67.0,51-100 feet,534.0,NaN,1960.0,1960-1969,NaN,2678.0,2678.0,14.0,2.0,0.0,NaN,Uncontrolled,NaN,NaN,NaN,NaN,NaN,NaN,Uncontrolled,2021-05-24,2017-01-24,2.0,High,Poor,2017-01-24,Yes,NaN,https://damsafety.org/georgia,NaN
43,Hightower Creek W/S Str. #18,Scataway Creek W/S Str # 18,NaN,GA00638,NaN,GA00638,Blue Ridge Mountain SWCD;Towns County Board of...,State;Local Government,State,0.0,Soil Conservation Service,No,Flood Risk Reduction,Flood Risk Reduction,Georgia,139-004-01662,34.956944,-83.672500,Georgia,Towns,Pine Grove Community,NaN,Scataway creek,"Congressional District 09, Georgia",NaN,Yes,Yes,GA-SAFE DAMS PROGRAM,Yes,Yes,Yes,NaN,NaN,Natural Resources Conservation Service,NaN,NaN,NaN,NaN,NaN,Earth,Earth,Unlisted/Unknown,Unlisted/Unknown,54.0,NaN,NaN,54.0,51-100 feet,700.0,NaN,1960.0,1960-1969,NaN,405.0,405.0,27.0,4.0,1.7,NaN,Uncontrolled,NaN,NaN,NaN,NaN,NaN,NaN,Uncontrolled,2021-05-24,2009-04-01,2.0,High,Not Rated,NaN,No,NaN,https://damsafety.org/georgia,NaN
45,Ellijay River WS Str # 9,NaN,NaN,GA00631,NaN,GA00631,Gilmer County Board of Commissoners;Limestone ...,State,State,0.0,SCS,No,Flood Risk Reduction,Flood Risk Reduction,Georgia,061-012-00647,34.780030,-84.376670,Georgia,Gilmer,NaN,NaN,ROCK CREEK,"Congressional District 09, G

In [ ]:
acre_feet = large_dams['Max Storage (Acre-Ft)'].mean()
acre_feet

117397.18224935731

In [ ]:
gallons = acre_feet * 326000
gallons

38271481413.29048